In [1]:
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))
tf.config.run_functions_eagerly(False)
print("Eager execution: {}".format(tf.executing_eagerly()))
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions

from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor

TensorFlow version: 2.6.0
Eager execution: True


In [2]:
# Building an identity block from scratch

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    # Retrive filters
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    # First component of main path
    
    X = Conv2D(F1, (1,1), strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis=3)(X, training = training)(X)
    X = Activation('relu')(X)
    
    # Second component of the main path
    
    X = Conv2D(F2, (f,f), strides = (1,1), padding ='same', kernel_initializer= initializer(seed = 0))(X)
    X = BatchNormalization(axis=3)(X, training = training)(X)
    X = Activation('relu')(X)
    
    # Third component of the main path
    
    X = Conv2D(F3, (1,1), padding='valid', strides = (1,1))(X)
    X = BatchNormalization(axis=3)(X, training = training)(X)
    
    
    # Add the skip connection
    
    out = Add()([X_shortcut, X])
    Y = Activation('relu')(out)
    
    return Y

In [3]:
# np.random.seed(1)
# X1 = np.ones((1,4,4,3)) * -1
# X2 = np.ones((1,4,4,3)) * 1
# X3 = np.ones((1,4,4,3)) * -1

# X = np.concatenate((X1,X2,X3), axis=0).astype(np.float32)

# A3 = identity_block(X, f=2, filters =[4,4,3], initializer=lambda seed=0:constant(value=1), training = False)

In [4]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))



TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable

In [33]:
## The convolution block
def convolutional_block(X, f, filters, s=2, training =True, initializer = random_uniform):
    # Retrieve the filters
    
    F1 , F2, F3 = filters
    
    X_shortcut = X
    
    # Main Path
    # First component
    X = Conv2D(F1, (1,1), strides=(s,s), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training = training)(X)
    X = Activation('relu')(X)
    
    # Second component
    X = Conv2D(F2, (1,1), strides=(1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training = training)(X)
    X = Activation('relu')(X)
    
    # Third component
    X = Conv2D(F3, (1,1), strides=(1,1), padding = "valid", kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training = training)(X)
    
    # Shortcut
    out = Conv2D(F3, (1,1), strides=(s,s), padding='valid', kernel_initializer= initializer(seed=0))(X_shortcut)
    out = BatchNormalization(axis=3)(X, training = training)(out)
    
    # Merge into the main path
    X = Add()([X, out])
    X = Activation('relu')(X)
    
    return X    

In [34]:
#from outputs import convolutional_block_output1, convolutional_block_output2
np.random.seed(1)
#X = np.random.randn(3, 4, 4, 6).astype(np.float32)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)
assert np.allclose(B.numpy(), convolutional_block_output2), "Wrong values when training=True."

print('\033[92mAll tests passed!')


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable

In [38]:
def ResNet50(input_shape=(64,64,3), classes = 6):
    # Define the input shaoe of the data
    X_input = Input(input_shape)
    
    # Zero Padding
    X = ZeroPadding2D((3,3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7,7), strides=(2,2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides = (2,2))(X)
    
    # Stage 2
    X = convolutional_block(X, f=3, filters = [64,64, 256], s=1)
    X = identity_block(X, 3, [64,64, 256])
    X = identity_block(X, 3, [64, 64,256])
    
    # Stage 3
    X = convolutional_block(X, f = 3, s=2, filters= [128, 128, 512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    
    # Stage 4
    X = convolutional_block(X, f = 3, s=2, filters= [256, 256, 1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    
    # Stage 5
    X = convolutional_block(X, f = 3, s=2, filters= [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    
    # Average Pooling
    X = AveragePooling2D((2,2))(X)
    
    # Flatten Layer
    X_flatten = Flatten()(X)
    X_dense = Dense(classes, activation='softmax', kernel_initializer= glorot_uniform(seed=0))(X_flatten)
    
    # Create model
    model = Model(inputs= X_input, outputs = X_dense)
    
    return model    

In [39]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

TypeError: 'KerasTensor' object is not callable

In [5]:
#### Transfer Learning with MobileNet v2

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation


In [7]:
# Create the datasets 
BATCH_SIZE = 32
IMG_SIZE = (160,160)
directory = 'dataset/'
train_dataset = image_dataset_from_directory(directory,
                                             shuffle = True,
                                             batch_size = BATCH_SIZE,
                                             image_size = IMG_SIZE,
                                             validation_split = 0.2,
                                             subset = 'training',
                                             seed = 42)
validation_dataset = image_dataset_from_directory(directory,
                                                  shuffle = True,
                                                  batch_size = BATCH_SIZE,
                                                  image_size = IMG_SIZE,
                                                  validation_split = 0.2,
                                                  subset = 'validation',
                                                  seed = 42)



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/'